In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np


In [2]:
pip install torch torchvision matplotlib Pillow


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.3 MB 1.8 MB/s eta 0:00:03
   --------- ------------------------------ 1.0/4.3 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.8/4.3 MB 2.2 MB/s eta 0:00:02
   --------------------- ------------------ 2.4/4.3 MB 2.4 MB/s eta 0:00:01
   ----------------------------- ---------- 3.1/4.3 MB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 3.9/4.3 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 2.9 MB/s  0:00:01
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/113.8 MB 3.6 MB/s eta 0:00:32
   ---------------------------------------- 1.3/113.8 MB 3.4 MB/s eta 0:00:34
    --------------------------------------- 2.1/113.8 MB 3.6 MB/s eta 0:00:32
   - -----------------------


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
dataset_path = r"D:\Users\Lenova\Desktop\YukTha\dataset"

train_dir = os.path.join(dataset_path, "train")
val_dir = os.path.join(dataset_path, "val")


In [7]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [9]:
from torchvision import datasets

In [10]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Classes:", train_dataset.class_to_idx)
print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))


Classes: {'ai': 0, 'real': 1}
Train size: 3000
Validation size: 600


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [15]:
model = models.mobilenet_v2(pretrained=True)

# Freeze feature layers
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier[1] = nn.Linear(model.last_channel, 2)

model = model.to(device)


c:\Users\Lenova\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenova\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-4)


In [9]:
num_epochs = 6

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_acc = 100 * correct / total
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Loss: {running_loss:.4f} "
          f"Train Accuracy: {train_acc:.2f}%")


Epoch [1/6] Loss: 108.6192 Train Accuracy: 71.40%
Epoch [2/6] Loss: 79.7148 Train Accuracy: 86.37%
Epoch [3/6] Loss: 65.2264 Train Accuracy: 89.13%
Epoch [4/6] Loss: 58.4274 Train Accuracy: 90.63%
Epoch [5/6] Loss: 53.4186 Train Accuracy: 91.13%
Epoch [6/6] Loss: 49.7494 Train Accuracy: 90.83%


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())
    

val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


NameError: name 'model' is not defined

In [14]:
torch.save(model.state_dict(), "models/mobilenet_food_model.pth")

print("Model saved successfully.")


Model saved successfully.


INFERENCE TIME CALCULATION


In [18]:
import time

model.eval()

sample_image, _ = next(iter(val_loader))
sample_image = sample_image[0].unsqueeze(0).to(device)

start = time.time()
with torch.no_grad():
    output = model(sample_image)
end = time.time()

print("Inference time (seconds):", end - start)


Inference time (seconds): 0.044744253158569336


Model load


In [17]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.last_channel, 2)

model.load_state_dict(torch.load(
    r"D:\Users\Lenova\Desktop\YukTha\models\mobilenet_food_model.pth",
    map_location=device
))

model = model.to(device)
model.eval()

print("Model loaded successfully.")


Model loaded successfully.


c:\Users\Lenova\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [22]:
import time
import numpy as np

model.eval()

times = []
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        start = time.time()
        outputs = model(images)
        end = time.time()
        
        times.append(end - start)
        
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    misclassified = []




avg_time = np.mean(times)
accuracy = correct / total

print("Average Batch Inference Time:", avg_time)
print("Per Image Inference Time:", avg_time / images.size(0))
print("Accuracy Check:", accuracy)


Average Batch Inference Time: 0.7333278467780665
Per Image Inference Time: 0.09166598084725831
Accuracy Check: 0.9166666666666666


In [28]:
misclassified = []
threshold = 0.6  # you can try 0.55, 0.6, 0.65

model.eval()

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        
        ai_probs = probs[:, 0]  # class 0 = ai
        
        # Correct threshold mapping
        preds = torch.where(
            ai_probs > threshold,
            torch.zeros_like(ai_probs).long(),   # Predict AI (0)
            torch.ones_like(ai_probs).long()     # Predict Real (1)
        )
        
        for i in range(len(labels)):
            if preds[i] != labels[i]:
                misclassified.append((labels[i].item(), preds[i].item()))

print("Total misclassified:", len(misclassified))
print("Accuracy:", 1 - len(misclassified)/len(val_dataset))


Total misclassified: 46
Accuracy: 0.9233333333333333


RISK SCORE FORMULA


NameError: name 'model' is not defined

In [1]:
pip install uvicorn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install "fastapi[standard]"


  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached markdown_it_py-4.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
   ---------------------------------------- 0.0/726.2 kB ? eta -:--:--
   -------------- ------------------------- 262.1/726.2 kB ? eta -:--:--
   ---------------------------------------- 726.2/726.2 kB 1.7 MB/s  0:00:00

   ----------------------------------------  0/22 [websockets]
   ----------------------------------------  0/22 [websockets]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- ------------------------------------  2/22 [sentry-sdk]
   --- -----------------------


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
probs = torch.softmax(outputs, dim=1)
ai_probs = probs[:, 0]  # since 0 = ai
preds = (ai_probs > 0.6).long()


In [11]:
print(val_dataset.class_to_idx)


{'ai': 0, 'real': 1}
